<a href="https://colab.research.google.com/github/ikeavin/student-exam-performance/blob/main/PredictStudentPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import neccessary libraries and load the dataset.

In [44]:
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
import pandas as pd
import numpy as np
import pickle
import requests
import os
import io

df = pd.read_csv("https://raw.githubusercontent.com/ikeavin/student-exam-performance/main/StudentsPerformance.csv")
display(df)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


Convert gender, race/ethnicity, parental level of education, lunch, and test preparation course to dummy variables.

In [45]:
df = pd.get_dummies(df, prefix=['gender','race/ethnicity','parental level of education', 'lunch', 'test preparation course'])
display(df)

,math score,reading score,writing score,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,72,72,74,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
1,69,90,88,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0
2,90,95,93,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,47,57,44,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
4,76,78,75,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,88,99,95,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0
996,62,55,55,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1
997,59,71,65,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
998,68,78,77,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0


Split our dataframe into 4 numpy arrays. One contains all our input values(x), one has our math scores(y_math), one has our writing scores(y_writing), and the final one has our reading scores(y_reading). 

In [46]:
# Remove the index column from our dataframe
df.reset_index(drop=True, inplace=True)

# Separate the input values from the output values by dropping output columns
# and creating a numpy array consisting of the remaining columns' values  
x_columns = df.columns.drop('math score').drop('writing score').drop('reading score')
x = df[x_columns].values

# Prepare our 3 output arrays by converting the respective columns to numpy arrays
y_math = df['math score'].values
y_writing = df['writing score'].values
y_reading = df['reading score'].values

Create our first model, which will be used to predict the math scores.

In [47]:
#Split our dataset into test and train sets
x_math_train, x_math_test, y_math_train, y_math_test = train_test_split(
    x, y_math, test_size = .25, random_state = 14)

#Create the neural network
math_model = Sequential()
math_model.add(Dense(25, input_dim = x_math_train.shape[1], activation = 'relu'))
math_model.add(Dense(10, activation='relu'))
math_model.add(Dense(1))
math_model.compile(loss = 'mean_squared_error', optimizer = 'adam')
math_monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                             verbose=1, mode='auto', patience=5, restore_best_weights=True)
math_model.fit(x_math_train, y_math_train, validation_data=(x_math_test,y_math_test),
               callbacks=[math_monitor],verbose = 2, epochs = 1000)




Epoch 1/1000
24/24 - 1s - loss: 4540.2275 - val_loss: 4713.9004
Epoch 2/1000
24/24 - 0s - loss: 4483.4429 - val_loss: 4639.9639
Epoch 3/1000
24/24 - 0s - loss: 4392.6831 - val_loss: 4519.6782
Epoch 4/1000
24/24 - 0s - loss: 4241.5107 - val_loss: 4317.9346
Epoch 5/1000
24/24 - 0s - loss: 3993.7200 - val_loss: 3997.0007
Epoch 6/1000
24/24 - 0s - loss: 3616.5723 - val_loss: 3528.1379
Epoch 7/1000
24/24 - 0s - loss: 3095.7527 - val_loss: 2908.5737
Epoch 8/1000
24/24 - 0s - loss: 2435.7542 - val_loss: 2180.1104
Epoch 9/1000
24/24 - 0s - loss: 1711.4644 - val_loss: 1425.9418
Epoch 10/1000
24/24 - 0s - loss: 1041.2036 - val_loss: 809.4868
Epoch 11/1000
24/24 - 0s - loss: 558.6865 - val_loss: 425.0369
Epoch 12/1000
24/24 - 0s - loss: 305.0086 - val_loss: 259.8100
Epoch 13/1000
24/24 - 0s - loss: 224.2572 - val_loss: 213.9010
Epoch 14/1000
24/24 - 0s - loss: 208.5074 - val_loss: 201.8380
Epoch 15/1000
24/24 - 0s - loss: 202.2252 - val_loss: 196.6742
Epoch 16/1000
24/24 - 0s - loss: 197.0403 - v

Create our second model, which will be used to predict reading scores.

In [48]:
#Split our dataset into test and train sets
x_reading_train, x_reading_test, y_reading_train, y_reading_test = train_test_split(
    x, y_reading, test_size = .25, random_state = 14)

#Create the neural network
reading_model = Sequential()
reading_model.add(Dense(25, input_dim = x_reading_train.shape[1] ,activation='relu'))
reading_model.add(Dense(10, activation='relu'))
reading_model.add(Dense(1))
reading_model.compile(loss = 'mean_squared_error', optimizer = 'adam')
reading_monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                                verbose=1, mode='auto', patience=5, restore_best_weights=True)
reading_model.fit(x_reading_train, y_reading_train, validation_data=(x_reading_test, y_reading_test),
                  callbacks=[reading_monitor], verbose = 2, epochs = 1000)






Epoch 1/1000
24/24 - 1s - loss: 4887.7153 - val_loss: 5002.0698
Epoch 2/1000
24/24 - 0s - loss: 4790.6758 - val_loss: 4865.5708
Epoch 3/1000
24/24 - 0s - loss: 4609.9609 - val_loss: 4612.6597
Epoch 4/1000
24/24 - 0s - loss: 4275.9556 - val_loss: 4146.8604
Epoch 5/1000
24/24 - 0s - loss: 3701.3506 - val_loss: 3425.1077
Epoch 6/1000
24/24 - 0s - loss: 2897.1270 - val_loss: 2512.4280
Epoch 7/1000
24/24 - 0s - loss: 1968.5365 - val_loss: 1551.8324
Epoch 8/1000
24/24 - 0s - loss: 1104.7186 - val_loss: 779.2203
Epoch 9/1000
24/24 - 0s - loss: 522.2879 - val_loss: 359.1942
Epoch 10/1000
24/24 - 0s - loss: 271.3732 - val_loss: 232.9651
Epoch 11/1000
24/24 - 0s - loss: 215.6127 - val_loss: 209.8654
Epoch 12/1000
24/24 - 0s - loss: 205.2982 - val_loss: 203.0139
Epoch 13/1000
24/24 - 0s - loss: 199.8506 - val_loss: 197.0660
Epoch 14/1000
24/24 - 0s - loss: 194.8874 - val_loss: 192.4719
Epoch 15/1000
24/24 - 0s - loss: 190.6272 - val_loss: 188.8624
Epoch 16/1000
24/24 - 0s - loss: 186.5034 - val_l

Create our third model, which will be used to predict writing scores.

In [49]:
#Split our dataset into test and train sets
x_writing_train, x_writing_test, y_writing_train, y_writing_test = train_test_split(
    x, y_writing, test_size = .25, random_state = 14)

#Create the neural network
writing_model = Sequential()
writing_model.add(Dense(25, input_dim = x_writing_train.shape[1] ,activation='relu'))
writing_model.add(Dense(10, activation='relu'))
writing_model.add(Dense(1))
writing_model.compile(loss = 'mean_squared_error', optimizer = 'adam')
writing_monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                                verbose=1, mode='auto', patience=5, restore_best_weights=True)
writing_model.fit(x_writing_train, y_writing_train, validation_data=(x_writing_test, y_writing_test),
                  callbacks=[writing_monitor], verbose = 2, epochs = 1000)

Epoch 1/1000
24/24 - 1s - loss: 4788.1743 - val_loss: 4842.1973
Epoch 2/1000
24/24 - 0s - loss: 4691.3242 - val_loss: 4720.8994
Epoch 3/1000
24/24 - 0s - loss: 4529.7896 - val_loss: 4505.3428
Epoch 4/1000
24/24 - 0s - loss: 4251.7529 - val_loss: 4147.7954
Epoch 5/1000
24/24 - 0s - loss: 3814.7571 - val_loss: 3604.6980
Epoch 6/1000
24/24 - 0s - loss: 3187.4373 - val_loss: 2865.1680
Epoch 7/1000
24/24 - 0s - loss: 2390.1331 - val_loss: 1996.6415
Epoch 8/1000
24/24 - 0s - loss: 1539.7065 - val_loss: 1179.5282
Epoch 9/1000
24/24 - 0s - loss: 831.0374 - val_loss: 599.1286
Epoch 10/1000
24/24 - 0s - loss: 412.7406 - val_loss: 322.4455
Epoch 11/1000
24/24 - 0s - loss: 257.4534 - val_loss: 244.3576
Epoch 12/1000
24/24 - 0s - loss: 221.7238 - val_loss: 227.3390
Epoch 13/1000
24/24 - 0s - loss: 211.0547 - val_loss: 218.7177
Epoch 14/1000
24/24 - 0s - loss: 203.5599 - val_loss: 211.3530
Epoch 15/1000
24/24 - 0s - loss: 196.3525 - val_loss: 204.8194
Epoch 16/1000
24/24 - 0s - loss: 190.2027 - val_

Now we try to predict what grades would be achieved by students from different backgrounds.

In [62]:
#gender_female	gender_male	race/ethnicity_group A	race/ethnicity_group B	race/ethnicity_group C	race/ethnicity_group D	race/ethnicity_group E	parental level of education_associate's degree	parental level of education_bachelor's degree	parental level of education_high school	parental level of education_master's degree	parental level of education_some college	parental level of education_some high school	lunch_free/reduced	lunch_standard	test preparation course_completed	test preparation course_none
student_privliged = np.array([[1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0]])
student_middle = np.array([[1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1]])
student_disadvantaged = np.array([[0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1]])

sp_reading = reading_model.predict(student_privliged);
sp_writing = writing_model.predict(student_privliged);
sp_math = math_model.predict(student_privliged);
print("Privileged student scores:")
print(f"Reading: {sp_reading}")
print(f"Writing: {sp_writing}")
print(f"Math: {sp_math}")

sm_reading = reading_model.predict(student_middle);
sm_writing = writing_model.predict(student_middle);
sm_math = math_model.predict(student_middle);
print("Middle class student scores:")
print(f"Reading: {sm_reading}")
print(f"Writing: {sm_writing}")
print(f"Math: {sm_math}")

sd_reading = reading_model.predict(student_disadvantaged);
sd_writing = writing_model.predict(student_disadvantaged);
sd_math = math_model.predict(student_disadvantaged);
print("Disadvantaged student scores:")
print(f"Reading: {sd_reading}")
print(f"Writing: {sd_writing}")
print(f"Math: {sd_math}")

Privileged student scores:
Reading: [[79.571075]]
Writing: [[80.57438]]
Math: [[69.88846]]
Middle class student scores:
Reading: [[70.59622]]
Writing: [[70.73301]]
Math: [[58.9823]]
Disadvantaged student scores:
Reading: [[59.83892]]
Writing: [[56.239124]]
Math: [[64.30093]]
